##ETL processes
This notebook develops the ETL process for each table before proceeding to complete the `etl.py` file to load the datasets.

In [1]:
!ls

ETL-processes.ipynb  Review-dataset.ipynb    __pycache__       data
README.md	     Star-schema-ERD.drawio  create-tables.py  sql_queries.py


In [2]:
!pip install psycopg2-binary

In [17]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [18]:
conn = psycopg2.connect(host="rosie.db.elephantsql.com", database="koxdgtnq", user="koxdgtnq", password="hyik6zNrT28enwS3KAMIhRtCMlG4J2Ez")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data/')

In [49]:
filepath = song_files[0]

In [50]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARBEBBY1187B9B43DB,NaN,NaN,"Gainesville, FL",Tom Petty,SOFFKZS12AB017F194,A Higher Place (Album Version),236.17261,1994


1: songs Table
Extract Data for Songs Table
Select columns for song ID, title, artist ID, year, and duration
Use df.values to select just the values from the dataframe
Index to select the first (only) record in the dataframe
Convert the array to a list and set it to song_data

In [51]:
num_songs, artist_id, artist_latitude, artist_longitude, artist_location, artist_name, song_id, title, duration, year = df.values[0]
song_data = song_data = [song_id, title, artist_id, year, duration]
song_data

['SOFFKZS12AB017F194',
 'A Higher Place (Album Version)',
 'ARBEBBY1187B9B43DB',
 1994,
 236.17261]

Insert Record into Song Table
Implement the song_table_insert query in sql_queries.py and run the cell below to insert a record for this song into the songs table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the songs table in the sparkify database.

In [52]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run test.ipynb to check that record was added to table.

Process log_data
In this part, you'll perform ETL on the second dataset, log_data, to create the time and users dimensional tables, as well as the songplays fact table.

Let's perform ETL on a single log file and load a single record into each table.

* Use the get_files function provided above to get a list of all log JSON files in data/log_data
* Select the first log file in this list
* Read the log file and view the data*

In [55]:
log_files = get_files('data/log_data/')

In [56]:
filepath = log_files[0]

In [57]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1543279932796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
1,Jimi Hendrix,Logged In,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Woodstock Inprovisation,200,1543280209796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
2,Building 429,Logged In,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Majesty (LP Version),200,1543280448796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
3,The B-52's,Logged In,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Love Shack,200,1543282396796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38
4,Die Mooskirchner,Logged In,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Frisch und g'sund,200,1543282717796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38


3: time Table
Extract Data for Time Table

* Filter records by NextSong action
* Convert the ts timestamp column to datetime
Hint: the current timestamp is in milliseconds
* Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
Hint: use pandas' dt attribute to access easily datetimelike properties.
* Specify labels for these columns and set to column_labels
* Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe

In [58]:
df = df[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1543279932796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
1,Jimi Hendrix,Logged In,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Woodstock Inprovisation,200,1543280209796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
2,Building 429,Logged In,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Majesty (LP Version),200,1543280448796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
3,The B-52's,Logged In,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Love Shack,200,1543282396796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38
4,Die Mooskirchner,Logged In,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Frisch und g'sund,200,1543282717796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38


In [59]:
t = pd.to_datetime(df['ts'], unit='ms') 
t.head()

0   2018-11-27 00:52:12.796
1   2018-11-27 00:56:49.796
2   2018-11-27 01:00:48.796
3   2018-11-27 01:33:16.796
4   2018-11-27 01:38:37.796
Name: ts, dtype: datetime64[ns]

In [60]:
time_data = []
for line in t:
    time_data.append([line, line.hour, line.day, line.week, line.month, line.year, line.day_name()])
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [61]:
time_df = pd.DataFrame.from_records(time_data, columns=column_labels)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-27 00:52:12.796,0,27,48,11,2018,Tuesday
1,2018-11-27 00:56:49.796,0,27,48,11,2018,Tuesday
2,2018-11-27 01:00:48.796,1,27,48,11,2018,Tuesday
3,2018-11-27 01:33:16.796,1,27,48,11,2018,Tuesday
4,2018-11-27 01:38:37.796,1,27,48,11,2018,Tuesday


Insert Records into Time Table
Implement the time_table_insert query in sql_queries.py and run the cell below to insert records for the timestamps in this log file into the time table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the time table in the sparkify database.

In [64]:
cur.execute("""
ALTER TABLE time
ALTER COLUMN weekday TYPE VARCHAR;
""")

In [65]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Bad pipe message: %s [b'\xb2j\xbdmx\xb6\xc4\t\xd7gG\x00\xaao\xd4y\xe7\x94 \x86\xc2?h\xe02z5L\xa2m\xc8\xad\x1a\x86\xfbw\xafb\x15\nD\xe41\x160\x10x\xb0\x1c\x96\x00\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xe6\xe5\xf4An-\x06\xf7^\x0b\x8a\r%\xd2xJ\xad~B\xa05\xbc']
Bad pipe message: %s [b'\x1d\xc8\xd5\xa4\xa4\xdb\xbe\xef\xbe\xff\xb1\xe7\x81\x9e\xffV\xbdk\x00\x00\xa2\xc0\x14\xc0\n\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19\x00:\x00\x89\xc0\x0f\xc0\x05\x005\x00\x84\xc0\x13\xc0\t\x003\x002\x001\x000\x00\x9a\x00\x99\x00\x98\x00

Run test.ipynb to check that record was added to table.

#4: users Table
Extract Data for Users Table
Select columns for user ID, first name, last name, gender and level and set to user_df

In [66]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,88,Mohammad,Rodriguez,M,paid
1,88,Mohammad,Rodriguez,M,paid
2,88,Mohammad,Rodriguez,M,paid
3,38,Gianna,Jones,F,free
4,38,Gianna,Jones,F,free


Insert Records into Users Table
Implement the user_table_insert query in sql_queries.py and run the cell below to insert records for the users in this log file into the users table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the users table in the sparkify database.

In [67]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run test.ipynb to check that record was added to table.

#5: songplays Table
Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the songplays table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.

* Implement the song_select query in sql_queries.py to find the song ID and artist ID based on the title, artist name, and duration of a song.
* Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to songplay_data

Insert Records into Songplays Table
* Implement the songplay_table_insert query and run the cell below to insert records for the songplay actions in this log file into the songplays table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the songplays table in the sparkify database.

In [68]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

IndexError: tuple index out of range

Run test.ipynb to check that record was added to table.

#Close Connection to Sparkify Database

In [ ]:
conn.close()

#Implement etl.py
Use what you've completed in this notebook to implement etl.py